<a href="https://colab.research.google.com/github/susantaghosh1/nlp-notebooks/blob/develop/Language_Modelling%5BMLM%2BCLM%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install datasets transformers[sentencepiece]
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install scipy sklearn
!pip install accelerate

In [2]:
%%bash
nvidia-smi
python --version
which nvcc
echo $PATH
echo $LD_LIBRARY_PATH


Fri Jul 29 11:37:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [4]:
import transformers,torch,accelerate
transformers.__version__,torch.__version__,accelerate.__version__

('4.21.0', '1.12.0+cu113', '0.11.0')

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModelForMaskedLM
import torch
model_name = 'distilbert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [6]:
tokenizer

PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

## Pre processing the data

In [7]:
from datasets import load_dataset
imdb_dataset = load_dataset("imdb")
imdb_dataset

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [8]:
imdb_dataset['train'][0]

{'label': 0,
 'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are f

Tokenize the whole corpus and add one key containing the word_ids of the tokens

In [9]:
def tokenize_text(batch_of_data):
  batch_encoding = tokenizer(batch_of_data['text'])
  word_ids = []
  for idx,each_input_id in enumerate(batch_encoding['input_ids']):
    word_ids.append(batch_encoding.word_ids(idx))
  batch_encoding['word_ids'] = word_ids
  return batch_encoding

In [10]:
tokenized_datasets = imdb_dataset.map(
    tokenize_text, batched=True, remove_columns=["text", "label"]
)

  0%|          | 0/25 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [11]:
tokenized_datasets,len(tokenized_datasets['train'][0]['word_ids']),len(tokenized_datasets['train'][0]['input_ids']),len(tokenized_datasets['train']['word_ids'])

(DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
}),
 363,
 363,
 25000)

Now, concatenate all the tokens and then chunk it to a context_length. Here, I have taken context_length of 128 tokens considering GPU memory of google colab

In [12]:
# let's take 3 sample and print out the number of tokens per review:
sample = tokenized_datasets['train'][:3]
for idx,each_sample in enumerate(sample['input_ids']):
  print(f"review {idx} has {len(each_sample)} tokens")
sample.keys(),

review 0 has 363 tokens
review 1 has 304 tokens
review 2 has 133 tokens


(dict_keys(['input_ids', 'attention_mask', 'word_ids']),)

In [13]:
concatenated_example ={}
for key in sample.keys():
  concatenated_example[key] = sum(sample[key],[])


In [14]:
len(sum(sample['input_ids'],[])),len(concatenated_example['input_ids'])

(800, 800)

In [15]:
[_ for _ in range(0,20,5)][:3]

[0, 5, 10]

In [16]:
# now let's chunk it to 128
chunked_sample ={}
chunk_size = 128
for i in range(0,len(concatenated_example['input_ids']),chunk_size):
  print(i,i+chunk_size)


0 128
128 256
256 384
384 512
512 640
640 768
768 896


In [17]:
temp_list = []
for k,v in concatenated_example.items():
  for i in range(0,len(concatenated_example['input_ids']),chunk_size):
    temp_list.append(v[i:i+chunk_size])
  chunked_sample[k] = temp_list
  temp_list = []

In [18]:
chunked_sample.keys(),len(chunked_sample['input_ids']),[len(_) for _ in chunked_sample['input_ids']]

(dict_keys(['input_ids', 'attention_mask', 'word_ids']),
 7,
 [128, 128, 128, 128, 128, 128, 32])

as we are seeing the last chunk is smaller than the chunk size. There are two options for this : either to drop the last chunk or to pad to the chunk size. I will drop the last chuk here.

In [19]:
800/128,(800//128)*128,800//128

(6.25, 768, 6)

In [20]:
chunk_size = 128
def concatenate_and_chunk(batch_of_data):
  concatenated_data = {}
  for key in batch_of_data.keys():
    concatenated_data[key] = sum(batch_of_data[key],[])
  total_length = len(concatenated_data['input_ids'])
  total_length = (total_length//chunk_size)*chunk_size # modify the total length to drop the last chunk
  chunked_data = {}
  for k,v in concatenated_data.items():
    temp_list = []
    for i in range(0,total_length,chunk_size):
      temp_list.append(v[i:i+chunk_size])
    chunked_data[k] = temp_list
  chunked_data ['labels'] = chunked_data['input_ids'].copy()
  return chunked_data

  
  

In [21]:
sample_data = tokenized_datasets['train'].shuffle(seed=42).select(range(3))

In [22]:
chunked_sample_data = sample_data.map(concatenate_and_chunk,batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [23]:
[len(_) for _ in sample_data['input_ids']],[len(_) for _ in chunked_sample_data['input_ids']],[len(_) for _ in chunked_sample_data['labels']],

([179, 159, 237], [128, 128, 128, 128], [128, 128, 128, 128])

In [24]:
lm_datasets = tokenized_datasets.map(concatenate_and_chunk,batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [25]:
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61291
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59904
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 122957
    })
})

Now one final thing is left, that is inserting [MASK] tokens at random positions in the inputs. Let's do that now

In [26]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer,mlm_probability=0.15)

In [27]:
"""data collator is basically a collation function , which will take list of samples aka list of dict and form the batch, 
which dataloaded can use during training"""
some_samples = [lm_datasets['train'][i] for i in range(2)]
for _ in some_samples:
  _.pop('word_ids')
  print(tokenizer.decode(_['input_ids'],))


[CLS] i rented i am curious - yellow from my video store because of all the controversy that surrounded it when it was first released in 1967. i also heard that at first it was seized by u. s. customs if it ever tried to enter this country, therefore being a fan of films considered " controversial " i really had to see this for myself. < br / > < br / > the plot is centered around a young swedish drama student named lena who wants to learn everything she can about life. in particular she wants to focus her attentions to making some sort of documentary on what the average swede thought about certain political issues such
as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes a

In [28]:
len(lm_datasets['train']['input_ids'])

61291

In [29]:
collated_samples = data_collator(some_samples)
collated_samples

{'input_ids': tensor([[  101,  1045, 12524,   103,  2572,  8025,  1011,  3756,  2013,  2026,
          2678,  3573,  2138,  1997,  2035,  1996,  6704,  2008,  5129,  2009,
          2043,  2009,  2001,  2034,  2207,  1999,  3476,  1012,  1045,  2036,
          2657,  2008,  2012,  2034,  2009,   103,  8243,  2011,  1057,  1012,
          1055,  1012,  8205,  2065,  2009,  2412,   103,  2000,  4607,  2023,
          2406,  1010,  3568,  2108,  1037,  5470,  1997,  3152,  2641,   103,
          6801,  1000,  1045,  2428,  2018,  2000,  2156,  2023,   103,  2870,
           103,  1026,  7987,  1013,  1028,   103,  7987,  1013,  1028,  1996,
          5436,   103,  8857,  2105,  1037,   103,  4467,   103,  3076,  2315,
         14229,  2040,  4122,  2000,  4553,  2673,  2016,  2064,  2055,   103,
           103,  1999,  3327,   103,  4122,  2000,  3579,   103,  3086,  2015,
          2000,  2437,  2070,  4066,  1997,  4516,  2006,  2054,  1996,  2779,
         25430, 14728,  2245,  2055,  

In [30]:
collated_samples['input_ids'].shape

torch.Size([2, 128])

In [31]:
for _ in collated_samples['input_ids']:
  print(tokenizer.decode(_))

[CLS] i rented [MASK] am curious - yellow from my video store because of all the controversy that surrounded it when it was first released in 1967. i also heard that at first it [MASK] seized by u. s. customs if it ever [MASK] to enter this country, therefore being a fan of films considered [MASK] controversial " i really had to see this [MASK] myself [MASK] < br / > [MASK] br / > the plot [MASK] centered around a [MASK] swedish [MASK] student named lena who wants to learn everything she can about [MASK] [MASK] in particular [MASK] wants to focus [MASK] attentions to making some sort of documentary on what the average swede thought about certain political issues such
as the vietnam war and race issues blouse the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics advantages she [MASK] [MASK] with her drama teacher, classmates, and married men [MASK] [MASK] br / > < br / > what kills me about i am curious - [MASK] [MASK] that 

In [32]:
import collections
import numpy as np
from transformers import default_data_collator
wwm_probability = 0.2

In [37]:
some_samples = [lm_datasets['train'][i] for i in range(2)]
type(some_samples[0]['input_ids']),some_samples[0].keys()

(list, dict_keys(['input_ids', 'attention_mask', 'word_ids', 'labels']))

In [39]:
from transformers import DataCollatorForWholeWordMask
data_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer)
collated_sample = data_collator(some_samples)

In [41]:
for _ in collated_samples['input_ids']:
  print(tokenizer.decode(_))

[CLS] i rented [MASK] am curious - yellow from my video store because of all the controversy that surrounded it when it was first released in 1967. i also heard that at first it [MASK] seized by u. s. customs if it ever [MASK] to enter this country, therefore being a fan of films considered [MASK] controversial " i really had to see this [MASK] myself [MASK] < br / > [MASK] br / > the plot [MASK] centered around a [MASK] swedish [MASK] student named lena who wants to learn everything she can about [MASK] [MASK] in particular [MASK] wants to focus [MASK] attentions to making some sort of documentary on what the average swede thought about certain political issues such
as the vietnam war and race issues blouse the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics advantages she [MASK] [MASK] with her drama teacher, classmates, and married men [MASK] [MASK] br / > < br / > what kills me about i am curious - [MASK] [MASK] that 

##Fine Tuning


To train a model we need :
1. tokenized input and output 
2. torch.utils Dataset and Dataloader
3. Loss function, optimizer, learning rate scheduler
4. Evaluation Loop
5. Saving the best model a

In [42]:
# downsample the data
train_size = 10_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

In [43]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
%%bash
git